# Functions to be used in implementations
This notebook is a collection of functions which can be used on implementations.  

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt  
import matplotlib.image as mpimg  
import matplotlib.colors as clr
import numpy as np 
import cv2 as cv
import os
from scipy import ndimage as nd


plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 

In [4]:
def read_imgs(folder_name):
    imgs = []
    files = []
    folder = folder_name

    for filename in sorted(os.listdir(folder)):
        img = cv.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            img = cv.resize(img, (480, 360))
            imgs.append(img)
            files.append(filename)

    #np.savetxt("../test_data/bases.txt", files, newline="\n", fmt = "%s")
    
    return imgs, files

In [88]:
def read_values(folder_name):
    values = []
    pack = np.loadtxt(folder_name,dtype={'names': ('filenames', 'values'),
                         'formats': ('S20', 'i4')})
    i = np.argsort(pack)
    pack = pack[i]
    for value in pack[['values']]:
        value = value[0]
        values.append(value)
    
    return values

In [5]:
def thresh_process(imgs):

    ret = []
    for img in imgs:
        
        gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        median = cv.medianBlur(gray, 3)
        kernel = np.ones((3,3),np.uint8)

        dilation = cv.morphologyEx(median, cv.MORPH_DILATE, kernel)
        closing = cv.morphologyEx(dilation, cv.MORPH_CLOSE, kernel)
        _, thresh = cv.threshold(closing, 250, 255, cv.THRESH_BINARY)
        
        ret.append(thresh)
        
    return ret


In [6]:
def saturation_process(imgs):
    
    ret = []
    for img in imgs:
        kernel = np.ones((3,3),np.uint8)
        gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

        hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
        hsv = hsv[:,:,1]

        dilation = cv.morphologyEx(hsv, cv.MORPH_OPEN, kernel)

        blur = cv.GaussianBlur(dilation, (3,3), 0)

        gaus = cv.Canny(blur,250,255)
        gaus = cv.dilate(gaus, kernel, iterations = 1)
        gaus = cv.erode(gaus, kernel, iterations = 1)
        
        ret.append(gaus)
        
    return ret

In [7]:
def morph_saturation(imgs):
    
    ret = []
    
    for img in imgs:
        kernel = np.ones((3,3),np.uint8)

        hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
        hsv = hsv[:,:,1]

        blur = cv.medianBlur(hsv, 5)
        gaus = cv.Canny(blur,250,255)
        gaus = cv.medianBlur(gaus, 1)
        gaus = cv.dilate(gaus, kernel, iterations = 1)
        
        ret.append(gaus)
        
    return ret

In [8]:
def find_contour(img, target):
    fidelity = False
    fidelityValue = 1.7
    detected = img.copy()
    _, c, h = cv.findContours(target, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    fidelityRange = 0
    if fidelity:
        maxArea = .0
        for i in c: # With images it is convenient to know the greater area
            area = cv.contourArea(i)
            if area > maxArea:
                maxArea = area
        fidelityRange = maxArea - (maxArea * fidelityValue) # If objects have same size it prevents false detection

    totalContours = 0

    br = []
    for i in range(len(c)):
        if h[0][i][3] == -1 and cv.contourArea(c[i]) >= fidelityRange:
            totalContours += 1
            approx = cv.approxPolyDP(c[i], 3, True)
            br.append(cv.boundingRect(approx))
    for b in br:
        cv.rectangle(detected, (b[0], b[1]), (b[0] + b[2], b[1] + b[3]), (255, 0, 0), 3) 
    
    return totalContours, detected

In [9]:
def blob_detection(img, target):
    imcopy = img.copy()

    params = cv.SimpleBlobDetector_Params()

    params.filterByColor = True
    params.blobColor = 0

    params.filterByArea = True
    params.minArea = 50

    params.filterByCircularity = True
    params.minCircularity = 0.785
    params.maxCircularity = 1.0

    params.filterByConvexity = True
    params.minConvexity = 0.4
    params.maxConvexity = 1.0

    params.filterByInertia = True
    params.minInertiaRatio = 0.1

    detector = cv.SimpleBlobDetector_create(params)

    keypoints = detector.detect(target)
    total = len(keypoints)

    im_with_keypoints = cv.drawKeypoints (imcopy, keypoints, np.array([]), (255,0,0), cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    return total, im_with_keypoints

In [10]:
def hough_circles(img, target):
    copy = img.copy()
    size = (np.max(target))/16
    circles = cv.HoughCircles(target, cv.HOUGH_GRADIENT, 1.5, size, param1=300, param2=25, minRadius=0,maxRadius=50)
    num = 0
    
    if circles is not None:
        num = len(circles[0])
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        for (x, y, r) in circles:
            cv.circle(copy, (x, y), r, (0, 0, 255), 2)
            cv.rectangle(copy, (x - 3, y - 3), (x + 3, y + 3), (0, 128, 255), -1)

    return num, copy

In [110]:
def f1_measure(base, total):
    
    tp = 0
    fp = 0
    fn = 0
    score = 0
    
    if base==total:
        tp = total
    elif total > base:
        tp = base
        fp = total - base
    elif total < base:
        tp = total
        fn = base - total
    score = tp / (tp + (0.5) * (fp + fn))
        
    return score
    
    

In [126]:
def example_implementation():
    imgs, files = read_imgs("../test_data")
    #np.savetxt("../test_data/bases.txt", files, newline="\n", fmt = "%s")
    values = read_values("../test_data/bases.txt")
    threshes = morph_saturation(imgs)
    finals = []
    totals = []
    scores = []
    for (img, thresh, file, value) in zip(imgs, threshes, files, values):
        total, final = blob_detection(img, thresh)
        score = f1_measure(value, total)
        scores.append(score)
        finals.append(final)
        totals.append(total)
        #fig, axs = plt.subplots(1,2)
        #axs[0].imshow(img, cmap = 'gray')
        #axs[0].set_title('Original image')
        #axs[1].imshow(final, cmap = 'gray')
        #axs[1].set_title('Image w/ detected objects')

        #print("Filename:" , file, "Detected objects:", total)

    save = zip(files, imgs, threshes, finals, values, totals, scores)
    print("Average success:", (np.sum(scores) / len(scores)))
    
    return save

In [127]:
save = example_implementation()
for file, img, thresh, final, value, total, score in save:
    print(file, total, value, score)

Average success: 0.6007864814230861
roi_1.png 0 4 0.0
roi_2.png 0 6 0.0
roi_3.png 0 17 0.0
screws_005.png 2 1 0.6666666666666666
screws_006.png 1 2 0.6666666666666666
screws_009.png 1 2 0.6666666666666666
screws_021.png 6 7 0.9230769230769231
screws_037.png 5 6 0.9090909090909091
screws_047.png 3 3 1.0
screws_075.png 5 5 1.0
screws_084.png 7 7 1.0
screws_375.png 5 6 0.9090909090909091
ting_roi.png 1 28 0.06896551724137931
